In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import contractions
from textblob import TextBlob
import spacy
import nltk

In [3]:
df = pd.read_csv('dataset/Hotel_Reviews.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df['Negative_Review'][0]

In [ ]:
df['Positive_Review'][0]

In [4]:
df['Negative_Review'] = df['Negative_Review'].apply(lambda x: x.lower())

In [5]:
df['Positive_Review'] = df['Positive_Review'].apply(lambda x: x.lower())

# Removal Of Stopwords

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hassa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [7]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

In [8]:
df['Clean_Negative_Review'] = df['Negative_Review'].apply(lambda x: remove_stopwords(x))

In [9]:
df['Clean_Postive_Review'] = df['Positive_Review'].apply(lambda x: remove_stopwords(x))

In [ ]:
df.head()

In [ ]:
df['Clean_Negative_Review'][0]

In [ ]:
df['Clean_Postive_Review'][0]

# Frequent Words

In [10]:
import collection
from collections import Counter
word_count = Counter()
for text in df['Clean_Negative_Review']:
    for word in text.split():
        word_count[word] += 1


word_count.most_common(20)

[('room', 176026),
 ('negative', 129447),
 ('hotel', 74709),
 ('breakfast', 58478),
 ('small', 49880),
 ('staff', 39512),
 ('nothing', 38768),
 ('rooms', 34802),
 ('would', 32291),
 ('could', 32079),
 ('bed', 29828),
 ('one', 28095),
 ('bit', 27546),
 ('bathroom', 26585),
 ('night', 24063),
 ('little', 22536),
 ('like', 22445),
 ('shower', 21290),
 ('good', 20821),
 ('us', 20225)]

In [11]:
FREQUENT_WORDS = set(word for (word, wc) in word_count.most_common(3))
def remove_freq_word(text):
    return " ".join([word for word in text.split() if word not in FREQUENT_WORDS])

In [12]:
def remove_specific_values(text):
    import re
    # Replace the specific values with an empty string
    return re.sub(r'\b0[0-9a-zA-Z]*\b', '', text).strip()



In [13]:
# Apply the function to the 'text' column
df['Clean_Negative_Review'] = df['Clean_Negative_Review'].apply(remove_specific_values)
df['Clean_Postive_Review'] = df['Clean_Postive_Review'].apply(remove_specific_values)

# Rare Words

In [14]:
RARE_WORDS = set(word for (word, wc) in word_count.most_common()[:-10:-1])
RARE_WORDS

{'aerothermal',
 'beda',
 'cinde',
 'complintary',
 'hommy',
 'immpossible',
 'lau',
 'tioned',
 'wardroom'}

In [ ]:
# 

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_word(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [ ]:
df['Stemed_Negative_Review'] = df['Clean_Negative_Review'].apply(lambda x: stem_word(x))

In [ ]:
df['Stemed_Negative_Review'][0]

In [ ]:
df.drop(columns='stemed_Review', inplace = True)

In [ ]:
df['Stemed_Postive_Review'] = df['Clean_Postive_Review'].apply(lambda x: stem_word(x))

In [ ]:
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


lematizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R": wordnet.ADV}

def lemmatize_words(text):
    pos_text = pos_tag(text.split())
    return " ".join([lematizer.lemmatize(word,wordnet_map.get(pos[0],wordnet.NOUN)) for word, pos in pos_text])

In [ ]:
wordnet.NOUN

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
df['Lemmatized_Negative_Review'] = df['Clean_Negative_Review'].apply(lambda x: lemmatize_words(x))

In [ ]:
df['Lemmatized_Positive_Review'] = df['Clean_Postive_Review'].apply(lambda x: lemmatize_words(x))

In [ ]:
df.head()

In [ ]:
df.to_csv('dataset/clean_reviews')

In [ ]:
df['Lemmatized_Positive_Review'][]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Lemmatized_Positive_Review'])

In [ ]:
tfidf_vectorizer.get_feature_names_out()

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
# nltk.download('punkt')
# df['tokens_postive_review'] = df['Lemmatized_Positive_Review'].apply(nltk.word_tokenize)

# df['tokens_str'] = df['tokens_postive_review'].apply(lambda x: ' '.join(x))

count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df['Clean_Negative_Review'])

tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(count_matrix)

In [ ]:
tfidf_matrix

In [ ]:
dense_tfidf_sample = tfidf_matrix[:5].toarray()

In [ ]:
tfidf_df = pd.DataFrame(dense_tfidf_sample, columns=count_vectorizer.get_feature_names_out())

In [ ]:
tfidf_df

In [16]:
count_vectorizer.get_feature_names_out()

array(['10', '100', '1000', ..., 'zzzz', 'zzzzzzz', 'zzzzzzzzzzzz'],
      dtype=object)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Calculate the significance of each feature (column)
# We use the mean tf-idf score across all documents
feature_significance = np.asarray(tfidf_matrix.mean(axis=0)).flatten()

# Define a threshold to filter out less significant features
threshold = 0.1
significant_indices = np.where(feature_significance > threshold)[0]

# Filter the TF-IDF matrix to keep only significant features
filtered_tfidf_matrix = tfidf_matrix[:, significant_indices]

# Get the feature names for the significant features
significant_feature_names = np.array(count_vectorizer.get_feature_names_out())[significant_indices]

# If you need the filtered matrix as a DataFrame
# import pandas as pd
# filtered_tfidf_df = pd.DataFrame(filtered_tfidf_matrix.toarray(), columns=significant_feature_names)

# Output the significant feature names and their mean scores for verification
print("Significant features and their mean TF-IDF scores:")
for feature, score in zip(significant_feature_names, feature_significance[significant_indices]):
    print(f"{feature}: {score}")

Significant features and their mean TF-IDF scores:
negative: 0.24843576507416015


In [20]:
filtered_tfidf_matrix

<515738x1 sparse matrix of type '<class 'numpy.float64'>'
	with 129431 stored elements in Compressed Sparse Row format>

In [ ]:
tf_clean_negative_review = tfidf_vec.fit_transform(df['Clean_Negative_Review'])

In [ ]:
 # to print vocabulary
print("Vocabulary:\n", tfidf_vec.vocabulary_)
print("\n\nFeatures:\n", tfidf_vec.get_feature_names_out())

In [ ]:
tf_clean_negative_review.toarray()

In [ ]:
df['combined'] = df['Clean_Negative_Review'] + ' ' + df['Clean_Postive_Review']

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
tfidf_matrix = vectorizer.fit_transform(df['Clean_Negative_Review'])

In [ ]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray().astype('float32'), columns=vectorizer.get_feature_names_out())

In [ ]:
# Output sparse matrix information
print(f"Sparse Matrix Shape: {tfidf_matrix.shape}")
print(f"Sparse Matrix Type: {type(tfidf_matrix)}")

# If you need to access specific rows or columns of the TF-IDF matrix
row_index = 0
column_index = 0
print(f"TF-IDF value at row {row_index}, column {column_index}: {tfidf_matrix[row_index, column_index]}")

# Save the sparse matrix to disk if needed
from scipy import sparse
sparse.save_npz('tfidf_matrix.npz', tfidf_matrix)

# To load the matrix back
loaded_tfidf_matrix = sparse.load_npz('tfidf_matrix.npz')
print(f"Loaded Sparse Matrix Shape: {loaded_tfidf_matrix.shape}")

In [ ]:
from scipy.sparse import save_npz, load_npz, csr_matrix

# Define the function to convert sparse matrix to DataFrame in chunks
def sparse_matrix_to_dataframe(sparse_matrix: csr_matrix, chunk_size: int = 10000) -> pd.DataFrame:
    n_rows, n_cols = sparse_matrix.shape
    feature_names = vectorizer.get_feature_names_out()
    
    # Initialize an empty DataFrame
    df = pd.DataFrame()
    
    for start_row in range(0, n_rows, chunk_size):
        end_row = min(start_row + chunk_size, n_rows)
        chunk = sparse_matrix[start_row:end_row].toarray().astype('float32')
        chunk_df = pd.DataFrame(chunk, columns=feature_names)
        df = pd.concat([df, chunk_df], ignore_index=True)
    
    return df

# Convert the loaded sparse matrix to a DataFrame
tfidf_df = sparse_matrix_to_dataframe(loaded_tfidf_matrix, chunk_size=10000)

print(tfidf_df)